## Importing libs

In [ ]:
# create fake datasets
from sklearn.datasets import make_moons

# ML lib for GNG model
from neupy.algorithms import GrowingNeuralGas

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
%matplotlib inline